In [5]:
# item은 품목
# coperation은 판매법인
# location은 지역
# supply는 유통물량
# price : 유통 품목을 kg당 가격

In [6]:
 import pandas as pd
 df = pd.read_csv('https://raw.githubusercontent.com/bigdataleeky/python/main/data/%EA%B0%80%EA%B2%A9%EC%98%88%EC%B8%A1%ED%95%99%EC%8A%B5%EB%8D%B0%EC%9D%B4%ED%84%B0.csv')
 df.head(3)

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0


범주형 데이터를 수치화 해서 모델 만들기
- 원핫 인코딩
- 레이블 인코딩

In [7]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [8]:
df['year'] =  df['timestamp'].dt.year
df['month'] =  df['timestamp'].dt.month
df['day'] =  df['timestamp'].dt.day

In [9]:
df = df.drop(columns=['ID',	'timestamp', 'supply(kg)'])

In [10]:
df.head(2)

,item,corporation,location,price(원/kg),year,month,day
0,TG,A,J,0.0,2019,1,1
1,TG,A,J,0.0,2019,1,2


In [11]:
y = df['price(원/kg)']
X = df.drop(columns=['price(원/kg)'])

In [12]:
# 범주형 데이터를 수치화
# 라벨인코딩 -> 원핫
from sklearn.preprocessing import LabelEncoder

In [13]:
# 라벨 인코딩
for col in ['item',	'corporation',	'location']:
  le = LabelEncoder()
  X[col] =le.fit_transform(X[col])

In [14]:
# one hot
import numpy as np
for col in ['item' ,	'corporation' ,	'location']:
  X = pd.concat((X, pd.get_dummies(X['item'],prefix=col)), axis=1)

In [15]:
X.head(2)

,item,corporation,location,year,month,day,item_0,item_1,item_2,item_3,...,corporation_0,corporation_1,corporation_2,corporation_3,corporation_4,location_0,location_1,location_2,location_3,location_4
0,4,0,0,2019,1,1,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
1,4,0,0,2019,1,2,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True


In [16]:
# 데이터 나누기
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.fit_transform(X_test)

In [17]:
# Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor

In [18]:
# 교차검증을 통해서 성능을 측정
from sklearn.model_selection import cross_validate, StratifiedKFold, cross_val_score
# from sklearn.
lr = LinearRegression()
# cv  = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
score = cross_validate(lr,X_train_scaled,y_train)  # 평가단위 MAE
score
# 사이킷런에서는 값이 클수록 좋은 모델로 판단기준

{'fit_time': array([0.07922339, 0.08183384, 0.12806678, 0.11925244, 0.12410903]),
 'score_time': array([0.00224161, 0.00159359, 0.01017594, 0.01205707, 0.0103147 ]),
 'test_score': array([0.38251328, 0.3842433 , 0.37571398, 0.37450374, 0.38179043])}

In [19]:
# 선형회귀모델로 학습과 검증데이터 의 결정 계수를 알아보자
lr.fit(X_train_scaled, y_train)
lr.score(X_train_scaled, y_train), lr.score(X_test_scaled, y_test)

(0.38001859694320306, -3.4285455026050136e+16)

In [20]:
max_value = df['price(원/kg)'].max()
print("가장 큰 값:", max_value)
min_value = df['price(원/kg)'].min()
print("가장 작은 값:", min_value)


가장 큰 값: 20909.0
가장 작은 값: 0.0


In [21]:
# Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor

In [22]:
# Base Code : LinearRegression
df.describe()

,price(원/kg),year,month,day
count,59397.000000,59397.000000,59397.000000,59397.000000
mean,1131.680674,2020.601445,6.320420,15.685489
std,2029.941445,1.201092,3.519942,8.804446
min,0.000000,2019.000000,1.000000,1.000000
25%,0.000000,2020.000000,3.000000,8.000000
50%,0.000000,2021.000000,6.000000,16.000000
75%,1519.000000,2022.000000,9.000000,23.000000
max,20909.000000,2023.000000,12.000000,31.000000


In [23]:
# 교차검증을 통해서 성능 측정
from sklearn.model_selection import cross_validate, cross_val_score, StratifiedKFold
lr = LinearRegression()
cv = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
score = cross_val_score(lr, X, y)     # 평가단위 MAE
score.mean()
# 보통 사이킷런에서는 값이 클수록 좋은 모델로 판단 기준을 세움
# 하지만 교차검증에서는 숫자 크기가 작을 수록 좋음

0.07003796884730555

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
lr.fit(X_train,y_train)
print(lr.score(X_test,y_test))
y_test_predic = lr.predict(X_test)
y_train_predic = lr.predict(X_train)
mean_absolute_error(y_train,y_train_predic), mean_absolute_error(y_test,y_test_predic)

0.37376528696055755


(1032.125169326077, 1031.570804135101)

In [25]:
# 사이킷 런에서 사용가능한 평가지표의 종
# import sklearn
# sklearn.metrics.get_scorer_names()

In [26]:
# 선형 모델을 비선형으로
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures()
X_train_scaled_poly = pf.fit_transform(X_train_scaled)
X_test_scaled_poly = pf.fit_transform(X_test_scaled)
lr = LinearRegression()
lr.fit(X_train_scaled_poly,y_train)
lr.score(X_train_scaled_poly,y_train), lr.score(X_test_scaled_poly,y_test)

(0.0006053204225207054, -4.297394666237806e+22)

In [27]:
ridge = Ridge()
ridge.fit(X_train_scaled_poly, y_train)
ridge.score(X_train_scaled_poly, y_train), ridge.score(X_test_scaled_poly, y_test)

(0.0007958437988219158, -0.0012452737649120138)

In [28]:
lasso = Lasso()
lasso.fit(X_train_scaled_poly, y_train)
lasso.score(X_train_scaled_poly, y_train), lasso.score(X_test_scaled_poly, y_test)

(0.0007864030332256355, -0.0010685107032779584)

In [29]:
kn = KNeighborsRegressor()
kn.fit(X_train_scaled_poly, y_train)
kn.score(X_train_scaled_poly, y_train),kn.score(X_test_scaled_poly, y_test)

(0.198979125405356, -0.1999761446420989)

In [30]:
dtr = DecisionTreeRegressor()
dtr.fit(X_train_scaled, y_train)
dtr.score(X_train_scaled, y_train),dtr.score(X_test_scaled, y_test)

(1.0, -1.324099758540925)

In [31]:
from sklearn.linear_model import SGDRegressor
sg = SGDRegressor(max_iter=100)
sg.fit(X_train_scaled,y_train)
sg.fit(X_train_scaled,y_train), sg.fit(X_test_scaled,y_test)

(SGDRegressor(max_iter=100), SGDRegressor(max_iter=100))

하이퍼 파라미터 튜닝
- 대상 모델
  - KNeighborsRegressor
  - DecisionTreeRegressor

In [32]:
from sklearn.model_selection import RandomizedSearchCV
knn_params = {
    'n_neighbors': [5,10,15],
    'weights':['uniform','distance'],
    'algorithm':['auto','ball_tree','kd_tree','brute'],
    'leaf_size':[10,30,50],
    'p':[1,2]   # 1은 맨해튼 거리, 2는 유클리드 거리
}
knn = KNeighborsRegressor()
rs = RandomizedSearchCV(knn, param_distributions = knn_params)
rs.fit(X_train_scaled, y_train)

RandomizedSearchCV(estimator=KNeighborsRegressor(),
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'leaf_size': [10, 30, 50],
                                        'n_neighbors': [5, 10, 15], 'p': [1, 2],
                                        'weights': ['uniform', 'distance']})

In [33]:
rs.best_score_

-0.06964498278463163

In [34]:
best_model_kn = rs.best_estimator_
best_model_kn.score(X_train_scaled,y_train), best_model_kn.score(X_test_scaled, y_test)

(0.06588006704536498, -0.06835514741502302)

In [37]:
# 결정 트리 하이퍼 파리미터 튜닝
dt = DecisionTreeRegressor()
dt_params = {
    'criterion':['friedman_mse', 'absolute_error','poisson','squared_error'],
    'max_depth': [None, 5, 10, 15],  # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],   # 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4],     # 리프 노드에 필요한 최소 샘플 수
    'max_features': ['sqrt', 'log2']  # 각 분할에서 고려할 최대 특성 수
}
rs = RandomizedSearchCV(dt, param_distributions = dt_params)
rs.fit(X_train_scaled, y_train)
print(f"best parameter : {rs.best_params_}")
best_model_kn = rs.best_estimator_
best_model_kn.score(X_train_scaled, y_train),best_model_kn.score(X_test_scaled, y_test)

best parameter : {'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 5, 'criterion': 'poisson'}


(0.0014944620615561233, -0.001710814637797986)

데이터 좀 더 가공하기
- 주차구하기
- 주차에 대한 누적
- 공휴일 변수 추적


In [43]:
# 주차 변수 추가
from datetime import datetime

df = pd.read_csv('https://raw.githubusercontent.com/bigdataleeky/python/main/data/%EA%B0%80%EA%B2%A9%EC%98%88%EC%B8%A1%ED%95%99%EC%8A%B5%EB%8D%B0%EC%9D%B4%ED%84%B0.csv')

def get_week_number(str_date = '2024-04-09'):
  date = datetime.strptime(str_date,'%Y-%m-%d')
  return date.isocalendar()[1]

y = df['price(원/kg)']

In [44]:
df['week'] = df['timestamp'].map(lambda x : get_week_number(x))

In [45]:
df['year'] = pd.to_datetime(df['timestamp']).dt.year
df['month'] = pd.to_datetime(df['timestamp']).dt.month
df['day'] = pd.to_datetime(df['timestamp']).dt.day

In [46]:
# 년도 별 주차를 누적
np.unique(df['year'])
# for idx, year in enumerate(np.unique(df['year'])):
#   print(idx,year)

array([2019, 2020, 2021, 2022, 2023], dtype=int32)

In [50]:
# 2019 52 2020 53 2021 53 2022 53
def recalc_week(row):
  if row['year'] == 2019:
    return row['week']
  elif row['year'] == 2020:
    return row['week']+52
  elif row['year'] == 2021:
    return row['week']+52+53
  elif row['year'] == 2022:
    return row['week']+52+53+53
  elif row['year'] == 2023:
    return row['week']+52+53+53+53

In [51]:
df['week_num'] = df[['year','week']].apply(lambda x: recalc_week(x) , axis=1)

In [53]:
# 공휴일이면 1 그렇지 않으면 0
import holidays  # 공휴일
def check_holiday(x):
  kr_h = holidays.KR()
  if x in kr_h:
    return 1
  else:
    return 0
df['holiday'] = df['timestamp'].apply(lambda x : check_holiday(x))

In [54]:
df.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),week,year,month,day,week_num,holiday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,1,2019,1,1,1,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,1,2019,1,2,1,0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,1,2019,1,3,1,0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,1,2019,1,4,1,0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,1,2019,1,5,1,0


In [55]:
df = df.drop(columns=['ID',	'timestamp','supply(kg)','price(원/kg)'])

In [56]:
df.head()

,item,corporation,location,week,year,month,day,week_num,holiday
0,TG,A,J,1,2019,1,1,1,1
1,TG,A,J,1,2019,1,2,1,0
2,TG,A,J,1,2019,1,3,1,0
3,TG,A,J,1,2019,1,4,1,0
4,TG,A,J,1,2019,1,5,1,0


In [57]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in ['item'	,'corporation',	'location',	'week',	'year',	'month'	,'day',	'week_num']:
  df[col] = le.fit_transform(df[col])

In [58]:
from sklearn.model_selection import  train_test_split
x_train,x_test,y_train,y_test = train_test_split(df,y,random_state=42)

In [61]:
from sklearn.neighbors import KNeighborsRegressor
kn = KNeighborsRegressor(n_neighbors=15)
kn.fit(x_train,y_train)
kn.score(x_train,y_train), kn.score(x_test,y_test)

(0.5514705649442342, 0.4661495785919828)